In [1]:
import pandas as pd
import numpy as np
import utils
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.model_selection import train_test_split
from imp import reload
import altair as alt

In [43]:
reload(utils)

<module 'utils' from '/Users/sidharthkapur/personal-workspace/rhna/utils.py'>

In [2]:
projections_df = utils.load_detailed_projections_df()

In [3]:
components_df = utils.load_population_projections_df()

In [4]:
components_df['County'] = components_df['County'] + ' County'

In [5]:
components_df['Birth rate'] = components_df['Births'] / components_df['Start Population']
components_df['Death rate'] = components_df['Deaths'] / components_df['Start Population']

In [6]:
alt.data_transformers.enable('default')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [7]:
sf_age_distributions = (
    projections_df.loc[lambda x: x['county'] == 'San Francisco County']
    .groupby(['year', 'agerc'])['perwt'].sum().reset_index()
)

In [8]:
sf_age_distributions

,year,agerc,perwt
0,2010,0,7971
1,2010,1,7425
2,2010,2,7220
3,2010,3,6684
4,2010,4,6319
...,...,...,...
5146,2060,96,2834
5147,2060,97,2344
5148,2060,98,1717
5149,2060,99,1393


In [12]:
sf_age_distributions.loc[lambda x: x['agerc'] <= 18].groupby('year')['perwt'].sum()

year
2010    116549
2011    119860
2012    122728
2013    125035
2014    127059
2015    129701
2016    132614
2017    135600
2018    138057
2019    142419
2020    144916
2021    147381
2022    149307
2023    151132
2024    152194
2025    152780
2026    152587
2027    152124
2028    151293
2029    149881
2030    147620
2031    145924
2032    143706
2033    141637
2034    139261
2035    136923
2036    134626
2037    132793
2038    130059
2039    129147
2040    128583
2041    128499
2042    128719
2043    128951
2044    129633
2045    130739
2046    132206
2047    134002
2048    136204
2049    138733
2050    141277
2051    143963
2052    146816
2053    149731
2054    152623
2055    155561
2056    158215
2057    160424
2058    162360
2059    163864
2060    165177
Name: perwt, dtype: int64

In [9]:
%config InlineBackend.figure_format = 'retina'
alt.Chart(
    sf_age_distributions.loc[
        lambda x: (x['year'] <= 2030) & (x['year'] % 2 == 0)
    ],
    title='Population Projections for San Francisco'
).mark_line().encode(
    x=alt.X('agerc', title='Age'),
    y=alt.Y('perwt', title='Number of people'),
    color=alt.Color('year', title='Year', legend=alt.Legend(format='')),
)

alt.Chart(...)

In [318]:
age_pop_indexed = sf_age_distributions.loc[
    lambda x: (x['year'] <= 2030) & (x['year'] % 2 == 0)
].copy()
age_pop_indexed['age_group'] = pd.cut(age_pop_indexed['agerc'], pd.interval_range(0, 101, freq=5, closed='left'))
age_pop_by_groups = age_pop_indexed.groupby(['year', 'age_group'])['perwt'].sum()

In [342]:
pd.Series(pd.interval_range(0, 101, freq=5, closed='left')).apply()

0       0-4
1       5-9
2     10-14
3     15-19
4     20-24
5     25-29
6     30-34
7     35-39
8     40-44
9     45-49
10    50-54
11    55-59
12    60-64
13    65-69
14    70-74
15    75-79
16    80-84
17    85-89
18    90-94
19    95-99
dtype: object

In [356]:
%config InlineBackend.figure_format = 'retina'
to_plot = (age_pop_by_groups / age_pop_by_groups.loc[2010]).reset_index()
to_plot['age_group'] = to_plot['age_group'].apply(
    lambda x: f'{x.left:2d}-{x.right - 1}'
)
alt.Chart(to_plot, width=650, title='Projected Change in San Francisco Population').mark_line().encode(
    x=alt.X('age_group', title='Age group', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('perwt', title='Number of people, index = 2010'),
    color=alt.Color('year', title='Year', legend=alt.Legend(format=''))
)

alt.Chart(...)

In [297]:
(age_pop_indexed / age_pop_indexed.loc[2010, :]).

perwt                                                              \
agerc       0         1         2         3         4         5         6     
year                                                                          
2010   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000   
2012   1.099611  1.181818  1.104848  1.076302  1.105396  1.086493  1.095853   
2014   1.108518  1.218586  1.189197  1.260323  1.218231  1.172987  1.197485   
2016   1.130724  1.224646  1.191136  1.260473  1.286438  1.349363  1.325289   
2018   1.105633  1.231515  1.222992  1.265111  1.287704  1.345651  1.402107   
2020   1.077280  1.405657  1.167590  1.295183  1.378541  1.425690  1.493882   
2022   1.017062  1.123906  1.159003  1.445990  1.319038  1.451670  1.562203   
2024   0.967633  1.060337  1.082964  1.171454  1.289761  1.584315  1.481305   
2026   0.914189  1.015354  1.047922  1.095452  1.183732  1.275617  1.426241   
2028   0.865262  0.943165  0.977147  1.045033  1.163792  1.200581  1.299116   
2030   0.832643  0.904108  0.927008  0.976062  1.074695  1.139261  1.290449   

                                     ...                                \
agerc       7         8         9    ...       91        92        93    
year                                 ...                                 
2010   1.000000  1.000000  1.000000  ...  1.000000  1.000000  1.000000   
2012   1.056776  1.007290  0.978995  ...  1.218115  1.212226  0.976744   
2014   1.121585  1.076449  1.059726  ...  1.500924  1.476355  1.231266   
2016   1.232458  1.195140  1.117808  ...  1.902033  1.851211  1.636951   
2018   1.472059  1.366355  1.247489  ...  2.073013  2.091119  2.149871   
2020   1.416176  1.390467  1.475251  ...  1.907579  2.079585  2.297158   
2022   1.527941  1.482430  1.417717  ...  2.195009  2.071511  1.990956   
2024   1.565435  1.582056  1.518904  ...  2.212569  2.159170  2.313953   
2026   1.703981  1.525981  1.572237  ...  2.006470  2.314879  2.337209   
2028   1.377611  1.467664  1.703744  ...  2.042514  2.325260  2.108527   
2030   1.303339  1.368411  1.380091  ...  2.203327  2.214533  2.191214   

                                                                             
agerc       94        95        96        97        98        99        100  
year                                                                         
2010   1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  
2012   0.835098  1.215909  1.094444  0.802920  0.804762  0.984127  0.737052  
2014   1.143722  1.102273  0.891667  1.021898  0.790476  0.357143  0.509960  
2016   1.364599  1.497727  1.177778  0.802920  0.442857  0.761905  0.402390  
2018   1.842663  2.045455  1.355556  1.014599  0.752381  0.555556  0.326693  
2020   2.164902  2.902273  2.172222  1.916058  1.133333  1.150794  0.613546  
2022   2.065053  3.018182  2.697222  2.879562  2.176190  2.428571  1.183267  
2024   2.060514  2.690909  2.561111  3.116788  2.838095  3.777778  2.434263  
2026   2.198185  2.965909  2.577778  2.715328  2.533333  3.960317  3.709163  
2028   2.266263  3.063636  2.758333  2.952555  2.728571  3.579365  4.159363  
2030   2.278366  2.902273  2.902778  3.255474  2.852381  3.888889  4.505976  

[11 rows x 101 columns]

In [250]:
components_df.loc[lambda x: x['County'] == 'San Francisco County']

,County,Start Year,Start Population,Births,Deaths,Net migrants,End Year,End Population,Birth rate,Death rate
1888,San Francisco County,2010,810369,8897,5727,7696,2011,821235,0.010979,0.007067
1889,San Francisco County,2011,821235,8778,5527,10086,2012,834572,0.010689,0.006730
1890,San Francisco County,2012,834572,9093,5747,7145,2013,845063,0.010895,0.006886
1891,San Francisco County,2013,845063,8949,5461,6036,2014,854587,0.010590,0.006462
1892,San Francisco County,2014,854587,9104,5634,6348,2015,864405,0.010653,0.006593
1893,San Francisco County,2015,864405,8996,5704,4584,2016,872281,0.010407,0.006599
1894,San Francisco County,2016,872281,9011,5766,4817,2017,880343,0.010330,0.006610
1895,San Francisco County,2017,880343,8665,5947,3557,2018,886618,0.009843,0.006755
1896,San Francisco County,2018,886618,8669,5839,-88,2019,889360,0.009778,0.006586
1897,San Francisco County,2019,889360,8609,6098,409,2020,892280,0.009680,0.006857


In [123]:
coefs_index = (
    projections_df[['sex', 'race7', 'agerc']]
    .drop_duplicates()
    .loc[lambda x: x['sex'] == 'FEMALE']
    .sort_values(['sex', 'race7', 'agerc'])
    .set_index(['sex', 'race7', 'agerc'])
    .index
)

In [203]:
def extract_features(df):
    return df
#     return df.groupby(['agerc'])

In [134]:
pd.Series(0, index=coefs_index).groupby(['agerc'])

sex     race7                                           agerc
FEMALE  American Indian or Alaska Native, Non-Hispanic  0        0
                                                        1        0
                                                        2        0
                                                        3        0
                                                        4        0
                                                                ..
        White, Non-Hispanic                             96       0
                                                        97       0
                                                        98       0
                                                        99       0
                                                        100      0
Length: 707, dtype: int64

In [204]:
coefs_df = (
    projections_df
    .loc[lambda x: x['year'] >= 2020]
    .groupby(['year', 'county']).apply(
        lambda g: (
            extract_features(
                g.set_index(['sex', 'race7', 'agerc'])
                .reindex(coefs_index, fill_value=0)
                ['perwt']
            ).values
        )
    )
)

In [205]:
regression_df = coefs_df.reset_index(name='inputs').merge(
    components_df[['County', 'Start Year', 'Start Population', 'Births', 'Deaths', 'Birth rate', 'Death rate']],
    left_on=['year', 'county'],
    right_on=['Start Year', 'County'],
)

In [206]:
inputs = np.array(regression_df['inputs'].tolist()) / regression_df['Start Population'].values[:, np.newaxis]

In [207]:
inputs.shape

(2320, 707)

In [208]:
train_counties, test_counties = train_test_split(regression_df['county'].unique(), train_size=0.5)

In [209]:
train_df = regression_df.loc[lambda x: x['county'].isin(train_counties)]
test_df  = regression_df.loc[lambda x: x['county'].isin(test_counties)]

In [210]:
def get_X_y(df):
    X = np.array(df['inputs'].tolist()) / df['Start Population'].values[:, np.newaxis]
    y = df['Birth rate'].values
    return X, y

In [213]:
train_X, train_y = get_X_y(train_df)
test_X, test_y = get_X_y(test_df)

In [241]:
# births_lm = Ridge(alpha=1)
births_lm = LinearRegression()
births_lm.fit(train_X, train_y)

LinearRegression()

In [242]:
births_lm.score(test_X, test_y)

0.2860798652607762

In [243]:
births_lm.predict(test_X)

array([0.0125226 , 0.00876444, 0.01077823, ..., 0.00852294, 0.00904816,
       0.01454218])

In [244]:
test_y

array([0.0080573 , 0.01002506, 0.00979752, ..., 0.00830262, 0.00933617,
       0.01455346])

In [245]:
results_df = pd.DataFrame({
    'predictions': births_lm.predict(test_X),
    'actuals': test_y,
    'year': test_df['year'],
})

In [248]:
alt.Chart(results_df).mark_circle().encode(
    x=alt.X('predictions', scale=alt.Scale(domain=[0, 0.02])),
    y=alt.Y('actuals', scale=alt.Scale(domain=[0, 0.02])),
    color='year'
)

alt.Chart(...)